In [ ]:
from Favorfit_image_to_text import clip_image_to_text

clip_model = clip_image_to_text.load_interrogator("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/image_to_text/clip", device="cuda")

In [ ]:
from glob import glob

tps = glob("/media/mlfavorfit/sdb/favorfit_templates/templates/*/*")
tps += glob("/media/mlfavorfit/sdb/favorfit_templates/templates_freepik/*/*")

In [ ]:
from PIL import Image
from tqdm import tqdm

caption_dict = {}
for tp in tqdm(tps, total=len(tps)):
    img = Image.open(tp)
    caption = clip_image_to_text.inference(img, clip_model, mode="fast")
    caption_dict[tp] = caption

In [15]:
import json

with open("./caption_dict.json", mode="r") as f:
    json_like = json.load(f)

In [36]:
texts = list(json_like.values())
texts = [text.split(", ") for text in texts]
main_sent = [text[0] for text in texts]
keywords = [text[1:] for text in texts]

In [30]:
keywords_flatten = []
for keyword in keywords: keywords_flatten += keyword

In [35]:
from collections import Counter
counter_keywords = Counter(keywords_flatten)
important_keywords = "natural materials :: high detail, behance. polished, minimalist furniture, soft zen minimalist, trending on textures. com, clean 3 d render, soft geometric 3d shapes, raytraced 3d set design, raytracing shadows, image on the store website, john pawson"

In [76]:
import torch
import clip

def gen_features(model, text):
    tokens = clip.tokenize([text]).to(device)
    text_features = model.encode_text(tokens)

    return text_features

def cosine_similarity(v1, v2):
    # 정규화
    v1_normalized = torch.nn.functional.normalize(v1, p=2, dim=-1)
    v2_normalized = torch.nn.functional.normalize(v2, p=2, dim=-1)

    # 내적 계산
    similarity = torch.matmul(v1_normalized, v2_normalized.transpose(0, 1))
    
    return similarity

def dist(caption_embedding, embeddings_tensor):
    # 코사인 유사도 계산
    similarity = cosine_similarity(caption_embedding, embeddings_tensor)

    return similarity


device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "ViT-L/14"
model, _ = clip.load(model_name, device=device)

In [59]:
with torch.no_grad():
    embeddings = [gen_features(model, sentence).to("cpu").squeeze() for sentence in main_sent]
embeddings_arr = torch.cat(embeddings)

In [100]:
from PIL import Image

img = Image.open("./image/kamil.jpg")
caption = clip_image_to_text.inference(img, clip_model, mode="simple")

In [101]:
with torch.no_grad():
    caption_embedding = gen_features(model, caption).to("cpu")

In [102]:
dist_tensor = dist(caption_embedding, embeddings_arr)

In [114]:
torch.argsort(dist_tensor, descending=True)[0][:50]

tensor([ 1873, 11528, 17520,  8665,  7951, 16123,  7077, 16894, 14176,  4894,
         9968, 18673,  1011,  1008, 14538,  5305,  3193, 12676, 19610,  2706,
         9019, 17832, 13041,   946,  3606,  9074,  5067, 17882, 14330,  7036,
         5029, 14297, 16087, 12004,  2423, 14430,  5176,  8242,  7146,  7046,
        16095,  4844, 14132, 15911,  6843,  3975, 13367,  8541, 13441, 17407])

In [119]:
main_sent[17407]

'a cup sitting on top of a table'